In [ ]:
# Load data and packages
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns
import datetime as datetime
from collections import Counter
import time
import seaborn as sns
from scipy import stats
import plotly.figure_factory as ff
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import classification_report
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

directory_path = os.getcwd() + "\\Data\\sorting_event_volumes_2023.csv"

df = pd.read_csv(directory_path)

In [ ]:
def fill_missing_events(df):
    df_filled_list = []
    
    for center in df['sorting_center_name'].unique():
        df_center = df[df['sorting_center_name'] == center]
        output_belts = df_center['output_belt'].unique()
        
        all_dates = pd.date_range(start=df_center['scanning_date'].min(), end=df_center['scanning_date'].max())
        
        all_combinations = pd.MultiIndex.from_product(
            [[center], all_dates, output_belts],
            names=['sorting_center_name', 'scanning_date', 'output_belt']
        )
        
        all_combinations_df = pd.DataFrame(index=all_combinations).reset_index()
        df_filled_center = pd.merge(all_combinations_df, df_center, 
                                    on=['sorting_center_name', 'scanning_date', 'output_belt'], 
                                    how='left')
        
        df_filled_center['no_of_events'] = df_filled_center['no_of_events'].fillna(0.0001)
        df_filled_list.append(df_filled_center)
    
    df_filled = pd.concat(df_filled_list, ignore_index=True)
    
    return df_filled

In [ ]:
# Data cleaning
print("Number of rows original dataset is: " + str(df.shape[0]))

df = df.loc[df["event_type"] == "LAJ", :]
df.drop(['event_location', 'input_belt', 'position'], axis=1, inplace = True)
df.dropna(inplace = True)
df['output_belt'] = df['output_belt'].astype(int)
df = df.groupby(['sorting_center_name', 'scanning_date', 'output_belt'], as_index = False)['no_of_events'].sum()
df['scanning_date'] = pd.to_datetime(df['scanning_date'])
df = fill_missing_events(df)

print("Number of rows cleaned dataset is: " + str(df.shape[0]))

In [ ]:
# Data preparation
df['day'] = df['scanning_date'].dt.day
#df['month'] = df['scanning_date'].dt.month
df['weekday'] = df['scanning_date'].dt.dayofweek + 1
df['week'] = df['scanning_date'].dt.isocalendar().week
df['week_of_month'] = (df['day'] - 1) // 7 + 1
#df['yearday'] = df['scanning_date'].dt.day_of_year
#df['yearday_sin'] = np.sin(df['yearday'] / 7 * 2 * np.pi)
#df['yearday_cos'] = np.cos(df['yearday'] / 7 * 2 * np.pi)

sorting_center_names = df["sorting_center_name"].unique()
df["sorting_center_name"].value_counts()

In [ ]:
# Descriptive statistics (Jelmer)

# For analysis exclude the null events
filtered_df = df[df['no_of_events'] >= 1]

### Outlier management

In [ ]:
# Step 1: Calculate Q1 (25th percentile) and Q3 (75th percentile)
Q1 = filtered_df['no_of_events'].quantile(0.25)
Q3 = filtered_df['no_of_events'].quantile(0.75)

# Step 2: Calculate IQR (Interquartile Range)
IQR = Q3 - Q1

# Step 3: Define the outlier bounds
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Step 4: Find the outliers
outliers = filtered_df[(filtered_df['no_of_events'] < lower_bound) | (filtered_df['no_of_events'] > upper_bound)]

# Display the outliers
outliers_list = outliers[['sorting_center_name', 'no_of_events', 'scanning_date']]
outliers_list.head()

fig = px.histogram(outliers_list, x="scanning_date", y="no_of_events", color="sorting_center_name", title="Outliers per Date", barmode='group')
fig.show()

### Day of the month

In [ ]:
# Study for the sorting center VANTAA

# Define the number of valid months for each day of the month
# Days 1-28 appear in all 12 months, day 29 appears in 11 months (February excluded),
# day 30 appears in 11 months (February excluded), and day 31 appears in only 7 months
# (months missing the 31st: April, June, September, November).
valid_months = {day: 12 for day in range(1, 29)}  # Days 1-28 in all 12 months
valid_months[29] = 11  # Day 29 appears in 11 months
valid_months[30] = 11  # Day 30 appears in 11 months
valid_months[31] = 7   # Day 31 appears in 7 months

# Filter data for the sorting center VANTAA
center_data = df[df['sorting_center_name'] == 'VANTAA']

# Group the data by day of the month and sum the total number of events (orders)
daily_volume = center_data.groupby('day')['no_of_events'].sum()

# Normalize the daily volume by dividing by the number of months in which each day exists
# This adjusts for the fact that not all days (like the 29th, 30th, 31st) occur in every month
normalized_volume = daily_volume / daily_volume.index.map(valid_months)

# Calculate the percentage of normalized volume for each day
# by dividing by the total normalized volume for the sorting center
total_normalized_volume = normalized_volume.sum()  # Sum of normalized volumes for all days
percentage = (normalized_volume / total_normalized_volume) * 100  # Percentage for each day

# Create a DataFrame to store the result for VANTAA
result = pd.DataFrame({'VANTAA': percentage})

# Plot the data for all days (1-31) in a single bar chart
result.plot(kind='bar', figsize=(12, 6), width=0.8)

# Add a title and labels to the plot
plt.title('Order Volume Distribution for VANTAA (Days 1 to 31)')
plt.xlabel('Day of Month')
plt.ylabel('Percentage of Normalized Volume')

# Set the x-axis labels to be horizontal for better readability
plt.xticks(rotation=0)

# Adjust the layout to make sure everything fits well
plt.tight_layout()
plt.show()

### Per chute (Tom)

In [ ]:
def chute_demand_per_weekday(df, sorting_centers):
    fig, axes = plt.subplots(2, 3, figsize=(18, 12), sharey=True)
    
    axes = axes.flatten()

    for i, sorting_center_name in enumerate(sorting_centers):
        df_sc = df[df["sorting_center_name"] == sorting_center_name]
        chutes = df_sc["output_belt"].unique()

        chute_PH_share_dict = {}

        for chute in chutes:
            df_chute = df_sc[df_sc["output_belt"] == chute]
            if df_chute.shape[0] <= 183:
                continue
            chute_PH_share_dict[chute] = df_chute.groupby('weekday')['no_of_events'].sum() / df_chute['no_of_events'].sum() * 100

        ax = axes[i]
        for chute, PH_data in chute_PH_share_dict.items():
            ax.plot(PH_data.index, PH_data.values, marker='o', label=chute)

        ax.set_xlabel('Weekday (1 = Monday, 7 = Sunday)')
        if i % 3 == 0:
            ax.set_ylabel('Percentage of Deliveries (%)')
        ax.set_title(f'{sorting_center_name} Distribution')

        ax.grid(True)

    plt.suptitle('Delivery Distribution per Weekday Across Chutes for Multiple Sorting Centers', y=1.03)
    plt.tight_layout()

    # Show the plot
    plt.show()

In [ ]:

def chute_demand_planning_horizon_per_month(df, sorting_centers):
    fig, axes = plt.subplots(2, 3, figsize=(18, 12), sharey=True)
    
    axes = axes.flatten()

    for i, sorting_center_name in enumerate(sorting_centers):
        df_sc = df[df["sorting_center_name"] == sorting_center_name]
        chutes = df_sc["output_belt"].unique()

        chute_PH_share_dict = {}

        for chute in chutes:
            df_chute = df_sc[df_sc["output_belt"] == chute]
            if df_chute.shape[0] <= 183:
                continue
            chute_PH_share_dict[chute] = df_chute.groupby('month')['no_of_events'].sum() / df_chute['no_of_events'].sum() * 100

        ax = axes[i]
        for chute, PH_data in chute_PH_share_dict.items():
            ax.plot(PH_data.index, PH_data.values, marker='o', label=chute)

        ax.set_xlabel('Planning horizon')
        if i % 3 == 0:
            ax.set_ylabel('Percentage of Deliveries (%)')
        ax.set_title(f'{sorting_center_name} Distribution')

        ax.grid(True)

    plt.suptitle('Delivery Distribution over Planning Horizon Across Chutes for Multiple Sorting Centers', y=1.03)
    plt.tight_layout()

    # Show the plot
    plt.show()

In [ ]:
chute_demand_per_weekday(df, sorting_center_names)
chute_demand_planning_horizon_per_month(df, sorting_center_names)